# Object Detection Training

Clone the tensorflow models repository if it doesn't already exist

In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2364, done.
remote: Counting objects: 100% (2364/2364), done.
remote: Compressing objects: 100% (2051/2051), done.
remote: Total 2364 (delta 582), reused 970 (delta 287), pack-reused 0
Receiving objects: 100% (2364/2364), 30.68 MiB | 23.22 MiB/s, done.
Resolving deltas: 100% (582/582), done.


 Install the Object Detection API

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1606870 sha256=7c1ada23a5d4233f29930a3ce7ecf121660986d77e7cd445b4185a1889c183b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-bd97_1oh/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.1-cp36-none-any.whl size=43734 sha256=ab8cbeac62ce6dc404f2a5ecc21803ae1f858476d264d690cdab3851657e952e
  Stored in directory: /root/.cache/pip/wheels/65/fe/90/20d6d6d97223d80d20cb390be636619c536edab5658c12bdba
  Created wheel for dill: filename=dill-0.3.1.1-cp36-none-any.whl size=78533 sha256=822f23b8181b3680c236e7d32587022d20ac28012af8e0133f43054f3ce7cd62
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for hdfs: filename=hdfs-2.5.8-cp36-none-any.whl size=33213 sha256=b1652543042fa97f0257e4929e620dc1ec7a5683c2c88f51faf1eeb332

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.26.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


Install Tensorflow 1

In [ ]:
!pip install -U --pre tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 41kB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 512kB 41.4MB/s 
     |████████████████████████████████| 3.8MB 36.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=0b4c0e52320d660a1b10c4474bd78b5625c547f0cd6930c416463d6fa1c7ee04
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement tensorflow-estimator<2.5.0,>=2.4.0rc0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing ins

In [ ]:
import functools
import json
import os
import tensorflow.compat.v1 as tf
from tensorflow.python.util.deprecation import deprecated


from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.legacy import trainer
from object_detection.utils import config_util

tf.logging.set_verbosity(tf.logging.INFO)
flags = tf.app.flags
FLAGS = flags.FLAGS

In [ ]:
@deprecated(None, 'Use object_detection/model_main.py.')
def main(_):
  
    configs = config_util.get_configs_from_pipeline_file('/content/drive/MyDrive/Colab Notebooks/Object_Detection_Training/faster_rcnn_inception_v2_pets.config')
    model_config = configs['model']
    train_config = configs['train_config']
    input_config = configs['train_input_config']
    
    model_fn = functools.partial(
      model_builder.build,
      model_config=model_config,
      is_training=True)

    def get_next(config):
    return dataset_builder.make_initializable_iterator(
        dataset_builder.build(config)).get_next()

    create_input_dict_fn = functools.partial(get_next, input_config)

    env = json.loads(os.environ.get('TF_CONFIG', '{}'))
    cluster_data = env.get('cluster', None)
    cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None
    task_data = env.get('task', None) or {'type': 'master', 'index': 0}
    task_info = type('TaskSpec', (object,), task_data)

  # Parameters for a single worker.
    ps_tasks = 0
    worker_replicas = 1
    worker_job_name = 'lonely_worker'
    task = 0
    is_chief = True
    master = ''

    if cluster_data and 'worker' in cluster_data:
    # Number of total worker replicas include "worker"s and the "master".
    worker_replicas = len(cluster_data['worker']) + 1
    if cluster_data and 'ps' in cluster_data:
    ps_tasks = len(cluster_data['ps'])

    if worker_replicas > 1 and ps_tasks < 1:
    raise ValueError('At least 1 ps task is needed for distributed training.')

    if worker_replicas >= 1 and ps_tasks > 0:
    # Set up distributed training.
    server = tf.train.Server(tf.train.ClusterSpec(cluster), protocol='grpc',
                             job_name=task_info.type,
                             task_index=task_info.index)
    if task_info.type == 'ps':
        server.join()
        return

    worker_job_name = '%s/task:%d' % (task_info.type, task_info.index)
    task = task_info.index
    is_chief = (task_info.type == 'master')
    master = server.target

    graph_rewriter_fn = None
    if 'graph_rewriter_config' in configs:
    graph_rewriter_fn = graph_rewriter_builder.build(
        configs['graph_rewriter_config'], is_training=True)

    trainer.train(
      create_input_dict_fn,
      model_fn,
      train_config,
      master='',
      task = 0,
      num_clones = 1,
      worker_replicas = 1,
      clone_on_cpu = False,
      ps_tasks = 0,
      worker_job_name = 'lonely_worker',
      is_chief = True,
      train_dir= '/content/drive/MyDrive/Colab Notebooks/Object_Detection_Training',
      graph_hook_fn=graph_rewriter_fn)

In [ ]:
if __name__ == '__main__':
    main(_)

# Training of the model starts

Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
INFO:tensorflow:Reading unweighted datasets: ['/content/drive/MyDrive/Colab Notebooks/Object_Detection_Training/Train.record']
INFO:tensorflow:Reading record datasets for input file: ['/content/drive/MyDrive/Colab Notebooks/Object_Detection_Training/Train.record']
INFO:tensorflow:Number of filenames to read: 1
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
Instructions for updating:
Use `tf.data.Dataset.map()
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
Instr

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession


Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Colab Notebooks/Object_Detection_Training/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt


...

INFO:tensorflow:global step 2621: loss = 0.5908 (0.183 sec/step)


INFO:tensorflow:global step 2621: loss = 0.5908 (0.183 sec/step)


INFO:tensorflow:global step 2622: loss = 1.2512 (0.196 sec/step)


INFO:tensorflow:global step 2622: loss = 1.2512 (0.196 sec/step)


INFO:tensorflow:global step 2623: loss = 0.5068 (0.193 sec/step)


In [ ]:
# TensorBoard

%load_ext tensorboard
%tensorboard --logdir '/content/training/train'